In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [167]:
def clean_df(df, suppr_str, keep_str):
    df = df.drop(columns=['Unnamed: 0'])
    df['date'] = df['date'].astype(str)
    columns = [x for x in df.columns.to_list() if '_norm' not in x and '_dis' not in x and 'std' not in x]
    columns = [x for x in columns if 'morning' in x or 'afternoon' in x or 'evening' in x or 'night' in x]
    
    for s in keep_str:
        columns = [x for x in columns if s in x]
    
    df = df[['pid','date'] + columns]
    
    for s in suppr_str:
        df.columns = [x.replace(s, '') for x in df.columns]

    df.columns = [x.replace(':morning', '_12:00').replace(':afternoon', '_18:00').replace(':evening', '_00:00').replace(':night', '_06:00') for x in df.columns]
    df.fillna(0, inplace=True)

    df = df.melt(id_vars=['pid', 'date'], value_vars=df.columns.to_list()[2:])
    df = df.sort_values(by=['pid', 'date', 'variable']).reset_index(drop=True)

    df['hour'] = df['variable'].apply(lambda x: x.split('_')[1]) 
    df['variable'] = df['variable'].apply(lambda x: x.split('_')[0])

    df['date'] = df['date'] + ':' + df['hour']
    df = df.drop(columns=['hour'])
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d:%H:%M')
    
    df = df.pivot(index=['pid', 'date'], columns='variable', values='value').reset_index()
    
    return df

In [168]:
df_steps = pd.read_csv('data/physionet.org/files/globem/1.1/INS-W_1/FeatureData/steps.csv')
df_sleep = pd.read_csv('data/physionet.org/files/globem/1.1/INS-W_1/FeatureData/sleep.csv')
df_screen = pd.read_csv('data/physionet.org/files/globem/1.1/INS-W_1/FeatureData/screen.csv')

/var/folders/51/b8ym03ns147cjmm4gt5_l48h0000gn/T/ipykernel_9042/367776588.py:3: DtypeWarning: Columns (808,809,810,811,812,813,814) have mixed types. Specify dtype option on import or set low_memory=False.
  df_screen = pd.read_csv('data/physionet.org/files/globem/1.1/INS-W_1/FeatureData/screen.csv')


In [169]:
df_steps = clean_df(df_steps, ['f_steps:fitbit_steps_intraday_rapids_', 'bout'], ['intraday'])
df_sleep = clean_df(df_sleep, ['f_slp:fitbit_sleep_intraday_rapids_', 'main', 'unified', 'within'], ['intraday'])
df_screen = clean_df(df_screen, ['f_screen:phone_screen_rapids_', '_', 'locmap'], [])

In [174]:
df_steps = df_steps[df_steps['pid'] == 'INS-W_001']
df_sleep = df_sleep[df_sleep['pid'] == 'INS-W_001']
df_screen = df_screen[df_screen['pid'] == 'INS-W_001']

In [182]:
df_steps.to_csv('data/physionet_cleaned/steps.csv', index=False)
df_sleep.to_csv('data/physionet_cleaned/sleep.csv', index=False)
df_screen.to_csv('data/physionet_cleaned/screen.csv', index=False)